In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
train = pd.read_csv('D:/Competetion/Kaggle/Titanic/train.csv')
test = pd.read_csv('D:/Competetion/Kaggle/Titanic/test.csv')
gender_submission = pd.read_csv('D:/Competetion/Kaggle/Titanic/gender_submission.csv')

In [3]:
train.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis = 1, inplace = True)
test.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis = 1, inplace = True)

In [4]:
# 删除带缺失值的变量
train = train.dropna()
test = test.dropna()

In [5]:
# 独热编码
train_dummy = pd.get_dummies(train[['Sex', 'Embarked']])
test_dummy = pd.get_dummies(test[['Sex', 'Embarked']])

In [6]:
# 拼接数据 
train_conti = pd.DataFrame(train, columns = ['Survived', 'Pclass', 'Age', 'Parch', 'Fare'], index = train.index)

train_conti = train_conti.join(train_dummy)
# test = test_std_conti.join(test_dummy)


In [7]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
889,1,1,male,26.0,0,0,30.0000,C


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    int64  
 1   Pclass    712 non-null    int64  
 2   Sex       712 non-null    object 
 3   Age       712 non-null    float64
 4   SibSp     712 non-null    int64  
 5   Parch     712 non-null    int64  
 6   Fare      712 non-null    float64
 7   Embarked  712 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 70.1+ KB


In [9]:
test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
409,3,female,3.0,1,1,13.7750,S
411,1,female,37.0,1,0,90.0000,Q
412,3,female,28.0,0,0,7.7750,S
414,1,female,39.0,0,0,108.9000,C


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331 entries, 0 to 415
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      331 non-null    float64
 1   Age         331 non-null    float64
 2   Parch       331 non-null    float64
 3   Fare        331 non-null    float64
 4   Sex_female  331 non-null    uint8  
 5   Sex_male    331 non-null    uint8  
 6   Embarked_C  331 non-null    uint8  
 7   Embarked_Q  331 non-null    uint8  
 8   Embarked_S  331 non-null    uint8  
dtypes: float64(4), uint8(5)
memory usage: 24.5 KB


In [19]:
x = train_conti.iloc[ : , 1: ]
y = train_conti.iloc[ : , 0 ]
# 使用iloc函数达到切片目的

##### 数据集划分

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)


In [21]:
x_train

,Pclass,Age,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
867,1,31.0,0,50.4958,0,1,0,0,1
332,1,38.0,1,153.4625,0,1,0,0,1
20,2,35.0,0,26.0000,0,1,0,0,1
606,3,30.0,0,7.8958,0,1,0,0,1
379,3,19.0,0,7.7750,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
885,3,39.0,5,29.1250,1,0,0,1,0
239,2,33.0,0,12.2750,0,1,0,0,1
791,2,16.0,0,26.0000,0,1,0,0,1
705,2,39.0,0,26.0000,0,1,0,0,1


In [22]:
y_train

867    0
332    0
20     0
606    0
379    0
      ..
885    0
239    0
791    0
705    0
858    1
Name: Survived, Length: 569, dtype: int64

In [23]:
# 归一化
stdsc = StandardScaler()
x_train_conti = stdsc.fit_transform(x_train[[ 'Pclass', 'Age', 'Parch', 'Fare']])
x_test_conti = stdsc.fit_transform(x_test[['Pclass', 'Age', 'Parch', 'Fare']])

# 标准化
x_train_conti_std = pd.DataFrame(x_train_conti, columns = ['Pclass', 'Age', 'Parch', 'Fare'], index = x_train.index)
x_test_conti_std = pd.DataFrame(x_test_conti, columns = ['Pclass', 'Age', 'Parch', 'Fare'], index = x_test.index)
#  抽离出所需的DataFrame

##### 模型训练

In [24]:
x_train

,Pclass,Age,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
867,1,31.0,0,50.4958,0,1,0,0,1
332,1,38.0,1,153.4625,0,1,0,0,1
20,2,35.0,0,26.0000,0,1,0,0,1
606,3,30.0,0,7.8958,0,1,0,0,1
379,3,19.0,0,7.7750,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
885,3,39.0,5,29.1250,1,0,0,1,0
239,2,33.0,0,12.2750,0,1,0,0,1
791,2,16.0,0,26.0000,0,1,0,0,1
705,2,39.0,0,26.0000,0,1,0,0,1


In [17]:
y_train

867    0
332    0
20     0
606    0
379    0
      ..
885    0
239    0
791    0
705    0
858    1
Name: Survived, Length: 569, dtype: int64

In [25]:
lgr = LogisticRegression(random_state = 0)
lgr.fit(x_train, y_train)


D:\Program\anoconda3\envs\TianChi\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=0)

In [27]:
x_test.shape

(143, 9)

In [29]:
y_test_predict = lgr.predict(x_test)
cm = confusion_matrix(y_test, y_test_predict)

In [30]:
cm

array([[73,  9],
       [17, 44]], dtype=int64)

In [32]:
print('logistic regression\'s scores is {:.2f}'.format(lgr.score(x_test, y_test)))
# logistic regression模型自带score函数

logistic regression's scores is 0.82
